In [0]:
# https://qiita.com/tomo_makes/items/f70fe48c428d3a61e131
# MNISTデータセットの準備
!pip install -q keras
import keras

In [5]:
# https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py
'''Trains a simple convnet on the MNIST dataset.

Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import numpy as np

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = np.abs(255 - x_train) # 白黒反転(白地に黒文字になるはず)
x_test = np.abs(255 - x_test)   # 白黒反転
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 12s 198us/step - loss: 0.2910 - acc: 0.9100 - val_loss: 0.0617 - val_acc: 0.9802
Epoch 2/12
60000/60000 [==============================] - 10s 164us/step - loss: 0.0937 - acc: 0.9716 - val_loss: 0.0452 - val_acc: 0.9855
Epoch 3/12
60000/60000 [==============================] - 10s 164us/step - loss: 0.0723 - acc: 0.9787 - val_loss: 0.0371 - val_acc: 0.9877
Epoch 4/12
60000/60000 [==============================] - 10s 164us/step - loss: 0.0605 - acc: 0.9823 - val_loss: 0.0333 - val_acc: 0.9889
Epoch 5/12
60000/60000 [==============================] - 10s 164us/step - loss: 0.0504 - acc: 0.9850 - val_loss: 0.0323 - val_acc: 0.9890
Epoch 6/12
60000/60000 [==============================] - 10s 164us/step - loss: 0.0458 - acc: 0.9864 - val_loss: 0.0314 - val_acc: 0.9895
Epoch 7/12
60000/60000 [===================

In [0]:
# 検証用 白地に黒文字か確認
from keras.preprocessing import image
import numpy as np
from google.colab import files

img = image.array_to_img(x_train[0], scale=True)
img.save("revert-x_train0.jpg", 'JPEG', quality=100, optimize=True)
files.download("revert-x_train0.jpg")
img = image.array_to_img(x_train[1], scale=True)
img.save("revert-x_train1.jpg", 'JPEG', quality=100, optimize=True)
files.download("revert-x_train1.jpg")

In [0]:
#モデルの保存
json_string = model.model.to_json()
open('./mnist_predict.json', 'w').write(json_string)
#重みの保存
hdf5_file = "./mnist_predict.hdf5"
model.model.save_weights(hdf5_file)

In [0]:
# ファイルをダウンロード ※ブラウザの上の方を見て、複数ファイルのダウンロードを許可する。
from google.colab import files
files.download("./mnist_predict.json")
files.download("./mnist_predict.hdf5")

In [15]:
# 0-9の手書き数字をまとめてアップロード
!rm *.zip *jpg *JPG
uploaded = files.upload()
!unzip -o num.zip
!ls -lh

rm: cannot remove '*.zip': No such file or directory
rm: cannot remove '*jpg': No such file or directory
rm: cannot remove '*JPG': No such file or directory


Saving num.zip to num.zip
Archive:  num.zip
  inflating: 0.jpg                   
  inflating: 1.jpg                   
  inflating: 2.jpg                   
  inflating: 3.jpg                   
  inflating: 4.jpg                   
  inflating: 5.jpg                   
  inflating: 6.jpg                   
  inflating: 7.jpg                   
  inflating: 8.jpg                   
  inflating: 9.jpg                   
total 28M
-rw-r--r-- 1 root root 1.3M Aug  9 12:49 0.jpg
-rw-r--r-- 1 root root 1.1M Aug  9 12:49 1.jpg
-rw-r--r-- 1 root root 1.2M Aug  9 12:49 2.jpg
-rw-r--r-- 1 root root 1.2M Aug  9 12:49 3.jpg
-rw-r--r-- 1 root root 1.2M Aug  9 12:49 4.jpg
-rw-r--r-- 1 root root 1.2M Aug  9 12:49 5.jpg
-rw-r--r-- 1 root root 1.2M Aug  9 12:49 6.jpg
-rw-r--r-- 1 root root 1.2M Aug  9 12:49 7.jpg
-rw-r--r-- 1 root root 1.3M Aug  9 12:49 8.jpg
-rw-r--r-- 1 root root 1.2M Aug  9 12:49 9.jpg
lrwxrwxrwx 1 root root    8 Aug 15 20:56 datalab -> /content
-rw-r--r-- 1 root root 4.6M Aug 22 

In [0]:
# # 画像の縮小検証
# !rm *-hw3*
# from keras.preprocessing import image
# import numpy as np
# from google.colab import files
# import cv2
# # 判定対象のファイルをアップロード
# uploaded = files.upload()
# !pwd
# !ls -l
# # input image dimensions
# img_rows, img_cols = 28, 28
# # 画像の読み込みと縮小 https://stackoverflow.com/questions/48121916/numpy-resize-rescale-image
# img_path = str(input())
# img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
# cv2.imwrite("GRAYSCALE-"+img_path, img)
# res = cv2.resize(img, dsize=(img_rows, img_cols), interpolation=cv2.INTER_NEAREST )
# cv2.imwrite("INTER_NEAREST-"+img_path, res)
# res = cv2.resize(img, dsize=(img_rows, img_cols), interpolation=cv2.INTER_LINEAR  )
# cv2.imwrite("INTER_LINEAR-"+img_path, res)
# res = cv2.resize(img, dsize=(img_rows, img_cols), interpolation=cv2.INTER_CUBIC  )
# cv2.imwrite("INTER_CUBIC-"+img_path, res)
# res = cv2.resize(img, dsize=(img_rows, img_cols), interpolation=cv2.INTER_AREA  )  
# cv2.imwrite("INTER_AREA-"+img_path, res)
# res = cv2.resize(img, dsize=(img_rows, img_cols), interpolation=cv2.INTER_LANCZOS4  )
# cv2.imwrite("INTER_LANCZOS4-"+img_path, res)
# !ls -lh
# !zip hw3.zip *-hw3*
# !ls -lh
# files.download("hw3.zip")

In [13]:
# https://qiita.com/tomo_20180402/items/e8c55bdca648f4877188  を参考にした
# 手書きの数字ファイルを選ばせる
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.models import model_from_json
import numpy as np
from google.colab import files
import cv2

#保存したモデルの読み込み
model = model_from_json(open('./mnist_predict.json').read())
#保存した重みの読み込み
model.load_weights('./mnist_predict.hdf5')

# 判定対象のファイルをアップロード
uploaded = files.upload()
!pwd
!ls -l

# input image dimensions
num_classes = 10
img_rows, img_cols = 28, 28

# 画像の読み込みと縮小
# https://stackoverflow.com/questions/48121916/numpy-resize-rescale-image
img_path = str(input())
img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
x = cv2.resize(img, dsize=(img_rows, img_cols), interpolation=cv2.INTER_AREA  ) 
# 検証用
cv2.imwrite("revert-"+img_path, x)
# files.download("revert-" + img_path)

# グレースケースを0以上1以下に変換
x = x.astype('float32')
x /= 255

# 軸を増やす
x = np.expand_dims(x, axis=0)
x = np.expand_dims(x, axis=3)

#予測
features = model.predict(x)

#予測結果
message = "Your writing number is: " + str(np.argmax(features))
print(message)
print("probability: " + str(np.max(features)*100) + " %")
print("Number : Probability")
for num, prob in enumerate(features[0]):
  # print(str(number) + " : " + str(features[0][number]) )
  print(str(num) + " : " + str(prob*100) + " %" )

Saving num.png to num.png
/content
total 4724
lrwxrwxrwx 1 root root       8 Aug 15 20:56 datalab -> /content
-rw-r--r-- 1 root root 4822552 Aug 22 07:24 mnist_predict.hdf5
-rw-r--r-- 1 root root    3667 Aug 22 07:24 mnist_predict.json
-rw-r--r-- 1 root root     858 Aug 22 07:26 num.png
drwxr-xr-x 2 root root    4096 Aug 15 20:56 sample_data
num.png
Your writing number is: 3
probability: 99.89011287689209 %
Number : Probability
0 : 4.554889798669137e-07 %
1 : 3.4547589677913493e-06 %
2 : 0.0439290190115571 %
3 : 99.89011287689209 %
4 : 5.391403234966674e-06 %
5 : 0.019417033763602376 %
6 : 1.9153077346345526e-06 %
7 : 0.00048486435844097286 %
8 : 0.013769684301223606 %
9 : 0.03228344430681318 %


In [16]:
# まとめて検証用
for num in range(0, 10):
  img_path = str(num) + ".jpg"
  print(img_path)
  # 画像の読み込みと縮小
  # https://stackoverflow.com/questions/48121916/numpy-resize-rescale-image
  img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
  x = cv2.resize(img, dsize=(img_rows, img_cols), interpolation=cv2.INTER_AREA  ) 
  # 検証用
  cv2.imwrite("revert-"+img_path, x)
  # files.download("revert-" + img_path)
  # グレースケースを0以上1以下に変換
  x = x.astype('float32')
  x /= 255
  
  # 軸を増やす
  x = np.expand_dims(x, axis=0)
  x = np.expand_dims(x, axis=3)

  #予測
  features = model.predict(x)

  #予測結果によって処理を分ける
  message = "Your writing number is: " + str(np.argmax(features))
  print(message)
  print("Possibility: " + str(np.max(features)))
  print(features)

0.jpg
Your writing number is: 5
Possibility: 0.1083215
[[0.10126617 0.10799599 0.10005082 0.10136531 0.09288523 0.1083215
  0.08984992 0.10151771 0.09484241 0.1019049 ]]
1.jpg
Your writing number is: 1
Possibility: 0.109674655
[[0.10151909 0.10967465 0.09952264 0.10219495 0.09209812 0.10921437
  0.08921852 0.1004088  0.09382995 0.10231891]]
2.jpg
Your writing number is: 1
Possibility: 0.1140582
[[0.08829714 0.1140582  0.10669201 0.11108937 0.08736827 0.11226936
  0.07631102 0.10936475 0.0904697  0.10408022]]
3.jpg
Your writing number is: 3
Possibility: 0.5247992
[[0.00228241 0.00516623 0.10807185 0.5247992  0.00312028 0.01190873
  0.00071824 0.09403665 0.01623191 0.23366453]]
4.jpg
Your writing number is: 4
Possibility: 0.15015174
[[0.07840432 0.14288808 0.09546856 0.09851012 0.15015174 0.0843795
  0.07272911 0.11600254 0.0721062  0.08935983]]
5.jpg
Your writing number is: 5
Possibility: 0.25683737
[[0.03759891 0.14972666 0.07201158 0.18259266 0.04955915 0.25683737
  0.04225325 0.07389

In [17]:
!zip revert.zip revert*jpg 
!ls -lh
files.download("revert.zip")

  adding: revert-0.jpg (deflated 2%)
  adding: revert-1.jpg (deflated 3%)
  adding: revert-2.jpg (deflated 2%)
  adding: revert-3.jpg (deflated 2%)
  adding: revert-4.jpg (deflated 2%)
  adding: revert-5.jpg (deflated 2%)
  adding: revert-6.jpg (deflated 3%)
  adding: revert-7.jpg (deflated 2%)
  adding: revert-8.jpg (deflated 2%)
  adding: revert-9.jpg (deflated 2%)
total 28M
-rw-r--r-- 1 root root 1.3M Aug  9 12:49 0.jpg
-rw-r--r-- 1 root root 1.1M Aug  9 12:49 1.jpg
-rw-r--r-- 1 root root 1.2M Aug  9 12:49 2.jpg
-rw-r--r-- 1 root root 1.2M Aug  9 12:49 3.jpg
-rw-r--r-- 1 root root 1.2M Aug  9 12:49 4.jpg
-rw-r--r-- 1 root root 1.2M Aug  9 12:49 5.jpg
-rw-r--r-- 1 root root 1.2M Aug  9 12:49 6.jpg
-rw-r--r-- 1 root root 1.2M Aug  9 12:49 7.jpg
-rw-r--r-- 1 root root 1.3M Aug  9 12:49 8.jpg
-rw-r--r-- 1 root root 1.2M Aug  9 12:49 9.jpg
lrwxrwxrwx 1 root root    8 Aug 15 20:56 datalab -> /content
-rw-r--r-- 1 root root 4.6M Aug 22 07:24 mnist_predict.hdf5
-rw-r--r-- 1 root root 3.6K A